<a href="https://colab.research.google.com/github/ysys143/ml2024/blob/main/BOK_QA_RAG_2_1_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Outline

Made by Jiwoo Kim, Jihoon Park, Jaesol Shin

## 환경설정

1) 실행 전 도구-명령팔레트-사용자 보안 비밀 탭 열기에서 HF_TOKEN의 값을 설정해주세요.

2) 지정된 위치에 폴더를 만들어 Q&A, 연차보고서 텍스트 파일을 넣어주세요.

path = '/content/drive/MyDrive/Google Playground/'

## Q&A
Encoder : Sentence-BERT 기반
Knowledge Source : BoK Q&A<br>
Finetuned by Q&A and 2019-2023 annaual reports
jiuuu26/chat-BOK -> roomnumber103/embedding-BOK<br>

## RAG
Knowledge Source : 2019-2023 annaual reports<br>
Chungking : RecursiveCharacterTextSplitter<br>
Retriever : Finetuned encoder<br>
VectorDB : FAISS<br>
LLM : Gemma2-2b-it 기반
Finetuned by Q&A and 2019-2023 annaual reports <br>
Parkppp/gemma2-BOK -> roomnumber103/LLM-BOK-gemma2
<br>

for more information : <a href="https://docs.google.com/document/d/1zHqC6-eiz2cPY8vopkAEKBnP9I_XkDPi2FSC3UCeqoo/edit?usp=sharing" >  BOK(Bank of Korea) Reports Analyzer</a>

In [ ]:
from google.colab import drive
from google.colab import userdata

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Google Playground/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 환경설정

In [ ]:
!pip install -q pandas numpy huggingface_hub langchain langchain-community faiss-cpu sentence-transformers torch accelerate datasets accelerate peft trl bitsandbytes pyarrow gradio

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from huggingface_hub import login
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Text Splitting
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.docstore.in_memory import InMemoryDocstore
from langchain import FAISS
import faiss

from langchain.prompts import PromptTemplate

import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer


from transformers import pipeline
from langchain import HuggingFacePipeline

from torch.amp import autocast

# Query-Answer Function

In [ ]:
# 사전정의된 Query-Answer가 담긴 테이블
qna_df = pd.read_csv(path + 'qa_data.csv')[['질문', '답변']]

qna_df['질문'] = qna_df['질문'].apply(lambda x: x.split('질문\n')[1]) # "질문\n" 제거
qna_df['답변'] = qna_df['답변'].apply(lambda x: x.split('답변\n')[1]) # "답변\n" 제거

In [ ]:
# SentenceTransformer 모델 로드
embedding_model = SentenceTransformer('roomnumber103/embedding-BOK')

# 쿼리 문장들에 대한 임베딩 벡터 생성
query_texts = qna_df['질문'].to_list()
query_embeddings = embedding_model.encode(query_texts)

In [ ]:
# query-answer 함수 정의
def qna_answer_to_query(new_query, embedding_model=embedding_model, query_embeddings=query_embeddings, top_k=1, verbose=True):
    # 쿼리 임베딩 계산
    new_query_embedding = embedding_model.encode([new_query])


    # 코사인 유사도 계산
    cos_sim = cosine_similarity(new_query_embedding, query_embeddings)

    # 코사인 유사도 값이 가장 큰 질문의 인덱스 찾기
    most_similar_idx = np.argmax(cos_sim)
    similarity = np.round(cos_sim[0][most_similar_idx], 2)

    # 가장 비슷한 질문과 답변 가져오기
    similar_query = query_texts[most_similar_idx]
    similar_answer = qna_df.iloc[most_similar_idx]['답변']

    if verbose == True:
        print("가장 비슷한 질문 : ", similar_query)
        print("가장 비슷한 질문의 유사도 : ", similarity)
        print("가장 비슷한 질문의 답: ", similar_answer)

    # 결과 반환
    return similar_query, similarity, similar_answer

# RAG Function

In [ ]:
# Document Loaders
text_folder = os.path.join(path, 'text_data')
#text_paths = glob.glob(os.path.join(path, '*.txt'))
text_paths = [os.path.join(text_folder, file) for file in os.listdir(text_folder)]

In [ ]:
documents = []

for i in range(len(text_paths)):
    print(text_paths[i])
    loader = TextLoader(text_paths[i])
    document_list = loader.load()  # document_list는 리스트 형식으로 반환될 수 있음

    # 리스트 안의 텍스트들을 하나의 문자열로 결합
    document_text = " ".join([doc.page_content for doc in document_list])

    # Document 객체로 변환하여 documents 리스트에 추가
    documents.append(Document(page_content=document_text))

# splitter 정의
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", "? ", "! "]
)

# Document 형식의 보고서를 splitter로 분할
splitted_documents = text_splitter.split_documents(documents)


# 문장이 마침표로 시작하면 마침표 제거, 마침표로 끝나지 않으면 마침표 추가
for i in range(len(splitted_documents)):
    splitted_documents[i].page_content = splitted_documents[i].page_content.strip('. ').strip()
    if not splitted_documents[i].page_content.endswith('.'):
        splitted_documents[i].page_content += '.'

/content/drive/MyDrive/Google Playground/text_data/rectified_text_2023.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2021.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2020.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2019.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2022.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2023.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2021.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2020.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2019.txt
/content/drive/MyDrive/Google Playground/text_data/rectified_text_2022.txt


In [ ]:
# 처음 FAISS Index를 생성시에는 아래 코드의 주석을 해제하여 사용한다
# Document 객체에서 텍스트 추출
report_texts = [doc.page_content for doc in splitted_documents]

# 문서 임베딩 생성 (embed_documents 대신 encode 사용)
#embeddings = embedding_model.encode(report_texts)

# FAISS 인덱스 생성
dim = 768  # 임베딩 차원 = len(embeddings[0])
#report_index = faiss.IndexFlatL2(dim)  # L2 거리 기반의 인덱스 생성
#report_index.add(np.array(embeddings))

# FAISS 인덱스 저장
#faiss.write_index(report_index, os.path.join(path, "report_index.index"))

# FAISS 인덱스 불러오기
loaded_index = faiss.read_index(os.path.join(path, "report_index.index"))

# 문서 저장소와 매핑 생성
docstore = InMemoryDocstore({idx: Document(page_content=text) for idx, text in enumerate(report_texts)})
docstore_id_map = {idx: idx for idx in range(len(report_texts))}

# embedding_function을 lambda로 수정하여 encode 호출
database = FAISS(embedding_function=lambda texts: embedding_model.encode(texts), index=loaded_index, docstore=docstore, index_to_docstore_id=docstore_id_map)

# Query Augmentation

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float32
)

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",]
)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline

# MPS 디바이스 설정
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

#모델 설정
model_id = "roomnumber103/LLM-BOK-gemma2"

# 모델 로드(Hub)
model = AutoModelForCausalLM.from_pretrained(model_id)

# 토크나이저 로드(Hub)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

# 모델 로드(local)
#model = AutoModelForCausalLM.from_pretrained(os.path.join(path, "gemma2_continued_lora/gemma2_continued_lora_model"))

# 토크나이저 로드(Local)
#tokenizer = AutoTokenizer.from_pretrained(os.path.join(path,"gemma2_continued_lora/gemma2_continued_lora_tokenizer"), use_fast=False)

# 모델에 동일한 PEFT (LoRA) 설정 적용
model = get_peft_model(model, peft_config)

# 디바이스 설정 및 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [ ]:
def question_gemma(prompt, model, tokenizer, temperature=0.7, max_new_tokens=512, line_length=80, max_sentences=5):

    # 입력 데이터 생성 및 이동
    # input_prompt = f"입력: {prompt}\n출력: "
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # 답변 생성
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=None,
            early_stopping=False
        )

    # 입력 부분을 제외한 생성된 토큰만 디코딩
    generated_tokens = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # 불필요한 '*' 제거
    response = re.sub(r"\*\*(.*?)\*\*", r"\1", response)

    return response

In [ ]:
# 프롬프트 템플릿 설정
prompt_template = PromptTemplate(
    template="""경제전문가로서 선행지식과 주어진 정보를 종합하여 답하시오. 1) 영어로 작성하지 말 것. Don't Use English. 2) 144자 정도의 완결된 문장으로 작성하고, 묻는 말에 끝까지 답할 것. 3) 답변에 **을 사용하지 말 것.

***Information***
{context}

***Question***
{query}

***Answer***""",
    input_variables=['context', 'query']
)

# 후처리 설정
def format_output_text(text, line_length=80):


    # 1. 문장 단위로 분리
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # 2. 각 문장들을 line_length마다 줄바꿈
    formatted_text = ""
    current_line = ""

    for sentence in sentences:
        # 현재 줄에 추가하여 줄 길이가 너무 길어지면 줄바꿈 추가
        if len(current_line) + len(sentence) < line_length:
            current_line += sentence + " "
        else:
            formatted_text += current_line.strip() + "\n"  # 줄바꿈 추가
            current_line = sentence + " "

    formatted_text += current_line.strip()  # 마지막 줄 추가
    return formatted_text


def trim_response(response, max_sentences=10):
    # 문장 단위로 자르기
    sentences = re.split(r'(?<=[.!?])\s+', response)
    # 최대 문장 수만큼만 유지
    return ' '.join(sentences[:max_sentences])

# 컨텍스트를 가져와 gemma에서 결과를 받은 뒤 형식을 다듬는 함수
def chat_with_retrieval(query, model=model, tokenizer=tokenizer, temperature=0.7, max_new_tokens=512, line_length=80, max_sentences=5):

    # 유사도가 높은 문장을 검색
    results = database.similarity_search(query, k=5)

    #조회를 통해 얻은 정보를 저장
    context = "Relevant documents:\n"

    for i, result in enumerate(results):
        context += f"""
        -----------------------------------
        Doc {i+1}:
        {result.page_content}
        """

    # 프롬프트 템플릿 적용
    prompt = prompt_template.format(context=context, query=query)

    # LLM을 통해 답변 생성
    response = question_gemma(prompt=prompt, model=model, tokenizer=tokenizer, temperature=temperature, max_new_tokens=max_new_tokens)  # 여기서는 LLM 호출 결과가 문자열로 반환됩니다.

    # 최대 문장 수로 자르기
    response = trim_response(response, max_sentences=max_sentences)

    # 출력 포맷팅
    formatted_response = format_output_text(response, line_length=line_length)

    return formatted_response

# Combine

In [ ]:
def full_answer_to_query(new_query, model=model, tokenizer=tokenizer, critical_value=0.5, temperature=0.7, max_new_tokens=512, line_length=80, max_sentences=5):
    """
    주어진 질문에 대해 Q&A 데이터셋에서 유사도를 검사하고, 유사도가 낮을 경우 모델을 통해 답변 생성.
    """
    # Q&A 데이터셋에서 유사 질문 찾기
    qna_result = qna_answer_to_query(new_query, verbose=False)
    qna_similarity = qna_result[1]
    qna_answer = qna_result[2]

    if qna_similarity >= critical_value:
        # Q&A 데이터셋에 유사 질문이 있는 경우
        msg = 'Q&A에 등재된 내용을 기반으로 답변을 구합니다.\n\n'
        return msg + qna_answer
    else:
        # 유사 질문이 없는 경우 -> 모델을 통해 답변 생성
        msg = '연차보고서를 참고로 Gemma2 모델을 사용하여 답변을 생성합니다.\n\n'
        generated_answer = chat_with_retrieval(new_query, model=model, tokenizer=tokenizer, temperature=0.7, max_new_tokens=512, line_length=80, max_sentences=5)
        return generated_answer

In [ ]:
qna_answer_to_query('2023년 통화정책의 운용목표는?')

가장 비슷한 질문 :  통화정책 최종대부자
가장 비슷한 질문의 유사도 :  0.43
가장 비슷한 질문의 답:  최종대부자(lender of last resort)란 금융위기로 인해 개별 금융기관 또는 전체 금융시장에 유동성 부족사태가 발생했을 때 위기 극복을 위해 유동성을 공급해 줄 수 있는 마지막 보루를 뜻합니다. 이는 현실적으로 화폐의 독점적 발행권과 무제한 공급능력을 가지고 있는 중앙은행만이 할 수 있는 일입니다.
  금융기관, 특히 은행은 일반적으로 단기로 예금을 받아 장기로 대출을 해 주기 때문에 보유하고 있는 자산과 부채의 만기가 일치하지 않습니다. 은행은 고객의 예금인출 요구에는 즉각 부응해야 하지만 대출을 회수하는 데는 시간이 걸립니다. 이에 따라 은행은 예금인출 사태(bank-run)가 발생하는 경우 예금인출 요구에 응하지 못하는 유동성 위기에 처할 가능성이 높습니다. 더욱이 한 은행의 유동성 위기는 금융시스템 전체에 대한 불신으로 이어져 건전한 은행이라도 동반하여 위기에 처해질 수 있습니다. 따라서 중앙은행은 금융위기가 발생하였을 때 지불능력은 있으나 단기적으로 유동성이 부족하여 예금인출 요구에 응할 수 없는 금융기관(solvent but illiquid institutions)에 충분한 자금을 공급해 위기를 넘길 수 있게 도와줌으로써 사람들의 불안 심리를 안정시키고 위기의 확산을 방지하는 역할을 합니다.
  한편 글로벌 금융위기 극복과정에서는 중앙은행이 개별 금융기관을 넘어 금융시장 전체에 유동성을 공급하는 정책도 수행하였습니다. 이렇게 전통적인 최종대부자 기능을 보완하는 중앙은행의 정책이 필요하게 된 것은 금융시장이 고도화되면서 회사채나 금융채 발행을 통한 자금조달이 확대되었고 이들 채권이 거래되는 유통시장(secondary market)의 효율적인 작동이 금융시스템 안정에 매우 중요해졌기 때문입니다.
  그러나 중앙은행의 최종대부자 기능 수행은 금융기관으로 하여금 고수익-고위험 자산을 더욱 선호하도록 하고 예금자가 금융기관 경영 및 재무

('통화정책 최종대부자',
 0.43,
 '최종대부자(lender of last resort)란 금융위기로 인해 개별 금융기관 또는 전체 금융시장에 유동성 부족사태가 발생했을 때 위기 극복을 위해 유동성을 공급해 줄 수 있는 마지막 보루를 뜻합니다. 이는 현실적으로 화폐의 독점적 발행권과 무제한 공급능력을 가지고 있는 중앙은행만이 할 수 있는 일입니다.\n  금융기관, 특히 은행은 일반적으로 단기로 예금을 받아 장기로 대출을 해 주기 때문에 보유하고 있는 자산과 부채의 만기가 일치하지 않습니다. 은행은 고객의 예금인출 요구에는 즉각 부응해야 하지만 대출을 회수하는 데는 시간이 걸립니다. 이에 따라 은행은 예금인출 사태(bank-run)가 발생하는 경우 예금인출 요구에 응하지 못하는 유동성 위기에 처할 가능성이 높습니다. 더욱이 한 은행의 유동성 위기는 금융시스템 전체에 대한 불신으로 이어져 건전한 은행이라도 동반하여 위기에 처해질 수 있습니다. 따라서 중앙은행은 금융위기가 발생하였을 때 지불능력은 있으나 단기적으로 유동성이 부족하여 예금인출 요구에 응할 수 없는 금융기관(solvent but illiquid institutions)에 충분한 자금을 공급해 위기를 넘길 수 있게 도와줌으로써 사람들의 불안 심리를 안정시키고 위기의 확산을 방지하는 역할을 합니다.\n  한편 글로벌 금융위기 극복과정에서는 중앙은행이 개별 금융기관을 넘어 금융시장 전체에 유동성을 공급하는 정책도 수행하였습니다. 이렇게 전통적인 최종대부자 기능을 보완하는 중앙은행의 정책이 필요하게 된 것은 금융시장이 고도화되면서 회사채나 금융채 발행을 통한 자금조달이 확대되었고 이들 채권이 거래되는 유통시장(secondary market)의 효율적인 작동이 금융시스템 안정에 매우 중요해졌기 때문입니다.\n  그러나 중앙은행의 최종대부자 기능 수행은 금융기관으로 하여금 고수익-고위험 자산을 더욱 선호하도록 하고 예금자가 금융기관 경영 및 재무상태의 건전성에 대한 판단 없이 더 높은 금리를 제시하는 금융기

가장 비슷한 질문 :  통화정책 최종대부자
가장 비슷한 질문의 유사도 :  0.43
가장 비슷한 질문의 답:  최종대부자(lender of last resort)란 금융위기로 인해 개별 금융기관 또는 전체 금융시장에 유동성 부족사태가 발생했을 때 위기 극복을 위해 유동성을 공급해 줄 수 있는 마지막 보루를 뜻합니다. 이는 현실적으로 화폐의 독점적 발행권과 무제한 공급능력을 가지고 있는 중앙은행만이 할 수 있는 일입니다.
  금융기관, 특히 은행은 일반적으로 단기로 예금을 받아 장기로 대출을 해 주기 때문에 보유하고 있는 자산과 부채의 만기가 일치하지 않습니다. 은행은 고객의 예금인출 요구에는 즉각 부응해야 하지만 대출을 회수하는 데는 시간이 걸립니다. 이에 따라 은행은 예금인출 사태(bank-run)가 발생하는 경우 예금인출 요구에 응하지 못하는 유동성 위기에 처할 가능성이 높습니다. 더욱이 한 은행의 유동성 위기는 금융시스템 전체에 대한 불신으로 이어져 건전한 은행이라도 동반하여 위기에 처해질 수 있습니다. 따라서 중앙은행은 금융위기가 발생하였을 때 지불능력은 있으나 단기적으로 유동성이 부족하여 예금인출 요구에 응할 수 없는 금융기관(solvent but illiquid institutions)에 충분한 자금을 공급해 위기를 넘길 수 있게 도와줌으로써 사람들의 불안 심리를 안정시키고 위기의 확산을 방지하는 역할을 합니다.
  한편 글로벌 금융위기 극복과정에서는 중앙은행이 개별 금융기관을 넘어 금융시장 전체에 유동성을 공급하는 정책도 수행하였습니다. 이렇게 전통적인 최종대부자 기능을 보완하는 중앙은행의 정책이 필요하게 된 것은 금융시장이 고도화되면서 회사채나 금융채 발행을 통한 자금조달이 확대되었고 이들 채권이 거래되는 유통시장(secondary market)의 효율적인 작동이 금융시스템 안정에 매우 중요해졌기 때문입니다.
  그러나 중앙은행의 최종대부자 기능 수행은 금융기관으로 하여금 고수익-고위험 자산을 더욱 선호하도록 하고 예금자가 금융기관 경영 및 재무

('통화정책 최종대부자',
 0.43,
 '최종대부자(lender of last resort)란 금융위기로 인해 개별 금융기관 또는 전체 금융시장에 유동성 부족사태가 발생했을 때 위기 극복을 위해 유동성을 공급해 줄 수 있는 마지막 보루를 뜻합니다. 이는 현실적으로 화폐의 독점적 발행권과 무제한 공급능력을 가지고 있는 중앙은행만이 할 수 있는 일입니다.\n  금융기관, 특히 은행은 일반적으로 단기로 예금을 받아 장기로 대출을 해 주기 때문에 보유하고 있는 자산과 부채의 만기가 일치하지 않습니다. 은행은 고객의 예금인출 요구에는 즉각 부응해야 하지만 대출을 회수하는 데는 시간이 걸립니다. 이에 따라 은행은 예금인출 사태(bank-run)가 발생하는 경우 예금인출 요구에 응하지 못하는 유동성 위기에 처할 가능성이 높습니다. 더욱이 한 은행의 유동성 위기는 금융시스템 전체에 대한 불신으로 이어져 건전한 은행이라도 동반하여 위기에 처해질 수 있습니다. 따라서 중앙은행은 금융위기가 발생하였을 때 지불능력은 있으나 단기적으로 유동성이 부족하여 예금인출 요구에 응할 수 없는 금융기관(solvent but illiquid institutions)에 충분한 자금을 공급해 위기를 넘길 수 있게 도와줌으로써 사람들의 불안 심리를 안정시키고 위기의 확산을 방지하는 역할을 합니다.\n  한편 글로벌 금융위기 극복과정에서는 중앙은행이 개별 금융기관을 넘어 금융시장 전체에 유동성을 공급하는 정책도 수행하였습니다. 이렇게 전통적인 최종대부자 기능을 보완하는 중앙은행의 정책이 필요하게 된 것은 금융시장이 고도화되면서 회사채나 금융채 발행을 통한 자금조달이 확대되었고 이들 채권이 거래되는 유통시장(secondary market)의 효율적인 작동이 금융시스템 안정에 매우 중요해졌기 때문입니다.\n  그러나 중앙은행의 최종대부자 기능 수행은 금융기관으로 하여금 고수익-고위험 자산을 더욱 선호하도록 하고 예금자가 금융기관 경영 및 재무상태의 건전성에 대한 판단 없이 더 높은 금리를 제시하는 금융기

In [ ]:
def context_documents(query, k=5):
    contexts = [doc.page_content for doc in database.similarity_search('2023년 통화정책의 운용목표는?', k=5)]
    for i, context in enumerate(contexts):
        print(f'Document {i+1}:\n{context}\n')

context_documents('2023년 통화정책의 운용목표는?', k=5)

Document 1:
2023년 통화신용정책 운영방향.

Document 2:
한국은행은 중기적 시계에서 소비자 물가 상승률이 물가 안정 목표에 근접하도록 통화신용정책을 운영하고 있다. 2023년 중 소비자 물가 상승률은 국제 원자재 가격 하락, 통화 긴축에 따른 수요 측 물가 압력 완화 등의 영향으로 3.6%로 낮아졌으나 2022년에 이어 물가 안정 목표를 상회하였다.

Document 3:
나. 기준금리 한국은행은 2023년 중 성장세를 점검하면서 중기적 시계에서 물가 상승률이 목표 수준(2%)에서 안정될 수 있도록 기준금리를 긴축적인 수준에서 운용하였다.

Document 4:
이 과정에서 코로나19의 전개 및 주요국의 경기 상황 등을 점검하는 한편, 자산시장으로의 자금 쏠림, 가계 부채 증가 등 금융 불균형 누적에 보다 유의할 것이다.

통화정책 방향에서는 금융통화위원회가 다음 통화정책방향 결정 시까지 한국은행 기준금리를 현 수준(0.50%)에서 유지하여 통화정책을 운용하기로 하였다.

Document 5:
아울러 2023년 2월부터 2025년까지 단계적으로 100%로 인상하는 계획을 발표하였다.

감시 및 리스크 관리 금융시장 인프라에 대한 평가 한국은행은 지급결제 분야의 국제 기준에 의거하여 중요 지급결제시스템에 대한 평가를 실시하고 미비점에 대해서는 개선을 권고하고 있다.

Document 1:
2023년 통화신용정책 운영방향.

Document 2:
한국은행은 중기적 시계에서 소비자 물가 상승률이 물가 안정 목표에 근접하도록 통화신용정책을 운영하고 있다. 2023년 중 소비자 물가 상승률은 국제 원자재 가격 하락, 통화 긴축에 따른 수요 측 물가 압력 완화 등의 영향으로 3.6%로 낮아졌으나 2022년에 이어 물가 안정 목표를 상회하였다.

Document 3:
나. 기준금리 한국은행은 2023년 중 성장세를 점검하면서 중기적 시계에서 물가 상승률이 목표 수준(2%)에서 안정될 수 있도록 기준금리를 긴축적인 수준에서 운용하였다.

Document

In [ ]:
print(question_gemma('2023년 통화정책의 운용목표는?', model=model, tokenizer=tokenizer))



답변: 

2023년 통화정책의 운용목표는 국가 경제 성장 및 안정적인 금리 상승을 이루어내는 것입니다.  


좀 더 자세한 설명:

* 국가 경제 성장: 2023년에는 국내 경제 활성화를 위한 다양한 정책이 시행될 예정이며, 이는 경제 성장에 기여하는 중요한 요소일 것입니다.
* 안정적인 금리 상승: 높은 금리가 국민의 소비를 저해하고 투자를 방해할 수 있으므로, 통화 정책은 금리를 안정적으로 높여야 합니다.


추가적인 고려 사항:

* 전체적인 경제 상황: 국내외 경제 상황과 관련하여 통화 정책의 운용 목표와 전략을 조정해야 할 수도 있습니다.
* 금리 수준: 현재의 금리 수준과 미래의 금리 수준을 고려하여 통화 정책을 결정해야 합니다. 


결론적으로, 2023년 통화정책의 운용목표는 국가 경제 성장과 안정적인 금리 상승을 동시에 달성하는 것이며, 이를 위해서는 경제 상황 변화에 대한 적절한 대응과 지속적인 모니터링이 필요합니다.










답: 
- 국가 경제 성장 및 안전한 금융 시스템 유지: 
  * 국내 기업 활성화, 부채 관리, 불안정한 환경 완화 등을 목표로 하는 통화 정책은 국가 경제 성장과 안전한 금융 시스템 유지를 위한 필수적인 요소이다.


추가 설명:

* 국가 경제 성장: 통화 정책은 국가 경제 성장에 큰 영향을 미치며, 이를 통해 높은 수입, 고용 증가, 그리고 소비 활발화를 이끌어낼 수 있다.
* 안정한 금융 시스템 유지: 통화 정책은 금리 조절, 자본 이동, 투자 유동 등으로 금융 시스템의 안정성을 유지하는 데 중요한 역할을 한다.


결론적으로 2023년 통화정책의 운용목표는 국가 경제 성장 및 안전한 금융 시스템 유지라는 두 가지 주요 목표를 달성하기 위해 적극적으로 실행되고 있는 것이다. 









In [ ]:
print(full_answer_to_query('2023년 통화정책의 운용목표는?', model=model, tokenizer=tokenizer))



.
. .


In [ ]:
print(chat_with_retrieval('2023년 통화정책의 운용목표는?', model=model, tokenizer=tokenizer))

한국은행은 2023년 통화정책을 운용하면서 중기적 시계에서 소비자 물가 상승률이 물가 안정 목표에 근접하도록 하는 것이었다.



# Web Client

In [ ]:
def chat_with_full_answer_to_query(message, history):
    # 사용자의 질문에 대해 full_answer_to_query를 사용하여 답변 생성
    response = full_answer_to_query(message, model, tokenizer)

    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, response))

    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

def chat_with_full_qna(message, history):
    # 사용자의 질문에 대해 qna_answer_to_query를 사용하여 답변 생성
    response = qna_answer_to_query(message)

    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, response))

    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

def get_context_documents(message, history):
    # 사용자의 질문에 대해 context_documents를 사용하여 관련 문서를 반환
    response = context_documents(message, k=5)

    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, response))

    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

def chat_with_gemma(message, history):
        # 사용자의 질문에 대해 사전학습된 모델만을 사용하여 답변 생성
    response = question_gemma(message, model=model, tokenizer=tokenizer)

    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, response))

    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history

In [ ]:
import gradio as gr

# Gradio Chatbot 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()  # 대화 기록을 표시하는 컴포넌트
    msg = gr.Textbox(label="질문 입력")  # 질문 입력을 위한 텍스트 박스
    clear = gr.Button("대화 기록 초기화")  # 대화 기록 초기화 버튼

    # 대화가 시작될 때 실행할 동작 정의
    def clear_history():
        return []

    #msg.submit(chat_with_full_answer_to_query, inputs=[msg, chatbot], outputs=[chatbot, chatbot])
    msg.submit(chat_with_gemma, inputs=[msg, chatbot], outputs=[chatbot, chatbot])

    # 기록 초기화 버튼 동작 정의
    clear.click(clear_history, None, chatbot, queue=False)

# 앱 실행
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c1ebc28ec2ab753274.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
